In [136]:
import duckdb

In [145]:
# set the query
query = f""" SELECT a.player_name, b.season, b.fee, b.from_club, b.to_club, b.transfer_date, STRPTIME(b.transfer_date, '%d/%m/%Y')::DATE 
            FROM players a 
            LEFT JOIN transfer_details b ON a.player_id = b.player_id
            WHERE b.transfer_date IS NOT NULL AND b.transfer_date <> ''
            ORDER BY id desc; """

In [142]:
query = """ 
SELECT * FROM sequence_analysis
WHERE player_name == 'Danny Welbeck'
"""

In [137]:
query = """ 
DROP TABLE sequence_analysis;
"""

In [146]:
# get data from db
conn = duckdb.connect('../transfermarkt.db')
try:
    df = conn.sql(query).df()
    # conn.execute(query)
    # results = conn.execute(query).fetchall()
except Exception as e:
    print(f"Error: {e}")
finally:
    conn.close()

In [144]:
df['sequence_string'][0]

'Man Utd → Preston → Man Utd → Sunderland → Man Utd → Arsenal → Watford → Brighton'

In [149]:
df[df['player_name'] == 'Kylian Mbappe']

player_name season            fee      from_club        to_club  \
149  Kylian Mbappe  11/12  free transfer  AS Bondy Yth.  Clairef. Yth.   
150  Kylian Mbappe  13/14  free transfer  Clairef. Yth.   Monaco Youth   
151  Kylian Mbappe  15/16              -   Monaco Youth     Monaco U19   
152  Kylian Mbappe  15/16              -     Monaco U19         Monaco   
153  Kylian Mbappe  17/18  loan transfer         Monaco            PSG   
154  Kylian Mbappe  17/18    End of loan            PSG         Monaco   
155  Kylian Mbappe  18/19       €180.00m         Monaco            PSG   
156  Kylian Mbappe  24/25  free transfer            PSG    Real Madrid   

    transfer_date CAST(strptime(b.transfer_date, '%d/%m/%Y') AS DATE)  
149    01/07/2011                                         2011-07-01   
150    01/07/2013                                         2013-07-01   
151    01/07/2015                                         2015-07-01   
152    01/01/2016                                         2016-01-01   
153    31/08/2017                                         2017-08-31   
154    30/06/2018                                         2018-06-30   
155    01/07/2018                                         2018-07-01   
156    01/07/2024                                         2024-07-01

In [48]:
with open("../output/list_club.txt", "w") as f:
    for club in list(df['club_name'].sort_values(ascending = True).unique()):
        # remove whitespace
        club = club.strip()
        f.write(f"{club}\n")

KeyError: 'club_name'

In [ ]:
string_ = "default.png?lm=1457423031"
"default" in string_

True

In [ ]:
# get image id from url
def get_club_id(url):
    try:
        parts = url.split('/')
        # find the last part
        id_part = parts[-1]
        if "_" in id_part:
            club_id = id_part.split('_')[0]
            return club_id
        else:
            club_id = id_part.split('.')[0]   
            return club_id 
        return None
    except:
        return None

url = "https://tmssl.akamaized.net//images/wappen/head/148_123123123.png?lm=1732011953"
get_club_id(url)

'148'

In [ ]:
conn = duckdb.connect('../transfermarkt.db')
print("\nExtracting clubs from transfer_details...")
    
# Get clubs from 'from_club' column
from_clubs = conn.execute("""
    SELECT DISTINCT 
        from_club as club_name,
        from_club_image_url as logo_url
    FROM transfer_details
    WHERE from_club IS NOT NULL 
        AND from_club_image_url IS NOT NULL
""").fetchdf()

print(f"  Found {len(from_clubs)} clubs in 'from_club' column")

# Get clubs from 'to_club' column
to_clubs = conn.execute("""
    SELECT DISTINCT 
        to_club as club_name,
        to_club_image_url as logo_url
    FROM transfer_details
    WHERE to_club IS NOT NULL 
        AND to_club_image_url IS NOT NULL
""").fetchdf()

print(f"  Found {len(to_clubs)} clubs in 'to_club' column")

# Combine and deduplicate
import pandas as pd
all_clubs = pd.concat([from_clubs, to_clubs]).drop_duplicates(subset=['club_name'])

print(f"  Total unique clubs: {len(all_clubs)}")

conn.close()


Extracting clubs from transfer_details...
  Found 5288 clubs in 'from_club' column
  Found 4155 clubs in 'to_club' column
  Total unique clubs: 4832


In [ ]:
all_clubs.head()

club_name                                           logo_url
0       Spurs U18  https://tmssl.akamaized.net//images/wappen/hea...
1        Brighton  https://tmssl.akamaized.net//images/wappen/hea...
2   Nordsjaelland  https://tmssl.akamaized.net//images/wappen/hea...
3  Right to Dream  https://tmssl.akamaized.net//images/wappen/hea...
4        Atalanta  https://tmssl.akamaized.net//images/wappen/hea...

In [ ]:
df.query("player_id == '192279'")['player_img_url'].values[0]

'https://img.a.transfermarkt.technology/portrait/header/192279-1661855851.jpg?lm=1'

In [ ]:
df.query("player_id == '258919'").sort_values(by='season', ascending=True)

id player_id season            fee      from_club         to_club  \
17  18    258919  13/14              -  Man Utd Youth     Man Utd U18   
15  16    258919  15/16  loan transfer    Man Utd U21       Stockport   
14  15    258919  15/16    End of loan      Stockport     Man Utd U21   
13  14    258919  15/16  loan transfer    Man Utd U21       Stockport   
12  13    258919  15/16    End of loan      Stockport     Man Utd U21   
16  17    258919  15/16              -    Man Utd U18     Man Utd U21   
11  12    258919  16/17  loan transfer    Man Utd U23    Grimsby Town   
10  11    258919  16/17    End of loan   Grimsby Town     Man Utd U23   
9   10    258919  17/18  loan transfer    Man Utd U23      Shrewsbury   
8    9    258919  17/18    End of loan     Shrewsbury     Man Utd U23   
6    7    258919  18/19    End of loan      Sheff Utd     Man Utd U23   
7    8    258919  18/19  loan transfer    Man Utd U23       Sheff Utd   
5    6    258919  19/20  loan transfer    Man Utd U23       Sheff Utd   
4    5    258919  20/21    End of loan      Sheff Utd     Man Utd U23   
3    4    258919  20/21              -    Man Utd U23         Man Utd   
2    3    258919  22/23  loan transfer        Man Utd   Nott'm Forest   
1    2    258919  22/23    End of loan  Nott'm Forest         Man Utd   
0    1    258919  23/24        €17.50m        Man Utd  Crystal Palace   

   transfer_date                                from_club_image_url  \
17    01/07/2013  https://tmssl.akamaized.net//images/wappen/hea...   
15    12/01/2016  https://tmssl.akamaized.net//images/wappen/hea...   
14    20/02/2016  https://tmssl.akamaized.net//images/wappen/hea...   
13    24/03/2016  https://tmssl.akamaized.net//images/wappen/hea...   
12    31/05/2016  https://tmssl.akamaized.net//images/wappen/hea...   
16    01/07/2015  https://tmssl.akamaized.net//images/wappen/hea...   
11    31/08/2016  https://tmssl.akamaized.net//images/wappen/hea...   
10    03/02/2017  https://tmssl.akamaized.net//images/wappen/hea...   
9     10/07/2017  https://tmssl.akamaized.net//images/wappen/hea...   
8     31/05/2018  https://tmssl.akamaized.net//images/wappen/hea...   
6     31/05/2019  https://tmssl.akamaized.net//images/wappen/hea...   
7     01/07/2018  https://tmssl.akamaized.net//images/wappen/hea...   
5     26/07/2019  https://tmssl.akamaized.net//images/wappen/hea...   
4     31/07/2020  https://tmssl.akamaized.net//images/wappen/hea...   
3     01/08/2020  https://tmssl.akamaized.net//images/wappen/hea...   
2     02/07/2022  https://tmssl.akamaized.net//images/wappen/hea...   
1     31/05/2023  https://tmssl.akamaized.net//images/wappen/hea...   
0     31/08/2023  https://tmssl.akamaized.net//images/wappen/hea...   

                                    to_club_image_url  
17  https://tmssl.akamaized.net//images/wappen/hea...  
15  https://tmssl.akamaized.net//images/wappen/hea...  
14  https://tmssl.akamaized.net//images/wappen/hea...  
13  https://tmssl.akamaized.net//images/wappen/hea...  
12  https://tmssl.akamaized.net//images/wappen/hea...  
16  https://tmssl.akamaized.net//images/wappen/hea...  
11  https://tmssl.akamaized.net//images/wappen/hea...  
10  https://tmssl.akamaized.net//images/wappen/hea...  
9   https://tmssl.akamaized.net//images/wappen/hea...  
8   https://tmssl.akamaized.net//images/wappen/hea...  
6   https://tmssl.akamaized.net//images/wappen/hea...  
7   https://tmssl.akamaized.net//images/wappen/hea...  
5   https://tmssl.akamaized.net//images/wappen/hea...  
4   https://tmssl.akamaized.net//images/wappen/hea...  
3   https://tmssl.akamaized.net//images/wappen/hea...  
2   https://tmssl.akamaized.net//images/wappen/hea...  
1   https://tmssl.akamaized.net//images/wappen/hea...  
0   https://tmssl.akamaized.net//images/wappen/hea...